In [1]:
import pandas as pd
from pybliometrics.scopus import ScopusSearch, CitationOverview
from ipywidgets import *
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('hello.mplstyle')

from tkinter import Tk, filedialog
from IPython.display import clear_output, display

import os
from tqdm.notebook import tqdm

# функция выбора файла
def select_files(b):
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=False)    # List of selected files will be set button's file attribute.
    print(b.files)
    
# функция чтения файла    
def read_files(b):
    file = b.files

    if file.endswith('.csv'):
        df = pd.read_csv(file)
    elif file.endswith('.xlsx') or file.endswith('.xls'):
        df = pd.read_excel(file)
    else:
        print('Error: Not supported file type.')

    return df

    
def on_click3(change):
    global y, alist, blist, listc, begin, ac
    if (pys.value == ()):
        py1,py2 = '',''
        y = []
    elif (pys.value[0] == pys.value[1]):
        py1 = py2 = pys.value[0]
        y = [py1]
    else:
        py1 = pys.value[0] - 1
        py2 = pys.value[1] + 1
        y = [i for i in range (py1+1, py2-1)]
    ac = acs.value
    
    begin = pd.DataFrame([], columns=y)
    
    b = ''
    alist = []
    blist = []
    listc = []
    for i in confs.tolist():
        listc.append(i)
        ### НАСТРОЙКА QUEUE ДЛЯ ПОИСКА ###
        disallowed_characters = "()"            
        for character in disallowed_characters:
            i = i.replace(character, "")
        if (py1 == py2) and (py1+py2 != '') and (ac != ''):
            a = ('CONF ( {} ) AND  PUBYEAR = ' + str(py1) + ' AND AFFILCOUNTRY ( ' + str(ac) + ' )').format(i)
            b = ('CONF ( {} ) AND  PUBYEAR = ' + str(py1)).format(i)
        elif (ac != '') and (py1 != '') and (py2 != ''):
            a = ('CONF ( {} ) AND  PUBYEAR > ' + str(py1) + ' AND  PUBYEAR < ' + str(py2) + ' AND AFFILCOUNTRY ( ' + str(ac) + ' )').format(i)
            b = ('CONF ( {} ) AND  PUBYEAR > ' + str(py1) + ' AND  PUBYEAR < ' + str(py2)).format(i)
        elif (str(py1) + str(py2) + ac == ''):
            a = 'CONF ( {} )'.format(i)
        elif (py1 + py2 == '') and (ac != ''):
            a = ('CONF ( {} ) AND AFFILCOUNTRY ( ' + str(ac) + ' )').format(i)
            b = ('CONF ( {} )').format(i)
        elif (py1 == py2) and (py1+py2 != '') and (ac == ''):
            a = ('CONF ( {} ) AND  PUBYEAR = ' + str(py1)).format(i)
        elif (ac == '') and (py1 != '') and (py2 != ''):
            a = ('CONF ( {} ) AND  PUBYEAR > ' + str(py1) + ' AND  PUBYEAR < ' + str(py2)).format(i)
        
        
        alist.append(a)
        if (b != ''):
            blist.append(b)    
        ### *** НАСТРОЙКА QUEUE ДЛЯ ПОИСКА *** ###    
    
    display(gobut)

def on_click2(change):
    global confs
    confs = df[colsel.value]
    print("If not needed, don't touch")
    display(pys, acs, ready3)
    ready3.observe(on_click3, 'value')


# отслеживаем, нажата ли кнопка "я закончил"
def on_click1(change):
    try:
        if (browsebut.files is None == False) or (browsebut.files != ''):
            global df
            df = read_files(browsebut)
            display(df)
            # список названий столбцов
            global colsel
            colsel = Dropdown(options=df.columns, description='Select the column:')
            display(colsel, ready2)
            
            ready2.observe(on_click2, 'value')
            
        else:
            print('Please, choose the file')
    except (AttributeError, UnboundLocalError) as e:
        print('Please, choose the file')
        
def ScSearch(b):
    global aresp, aresc, bresp, bresc
    aresp = pd.DataFrame([])
    bresp = pd.DataFrame([])
    aresc = pd.DataFrame([])
    bresc = pd.DataFrame([])
    for i in tqdm(alist):
        s = ScopusSearch(i, subscriber = False)
        df = pd.DataFrame(pd.DataFrame(s.results))
        if df.empty != True:

            df['coverDate'] = pd.DatetimeIndex(df['coverDate']).year # меняем данные на формат .year

            # публикации #
            temp1 = pd.DataFrame(df.value_counts(subset=['coverDate']), columns=['count']) # считаем публикации
            temp1 = temp1.rename_axis(None).transpose() # переворачиваем, потому что костыли надо вертеть
            temp1.columns = temp1.columns.get_level_values(0) # лечим то, что программа считает, что это мультииндекс
            temp2 = begin.merge(temp1,how='outer') #объединяем результаты
            temp2.index=["{}".format(listc[alist.index(i)])] # подписываем конференцию
            aresp = pd.concat([aresp,temp2])
            #* публикации *#

            # цитирования #
            temp3 = df.groupby('coverDate').sum('citedby_count').drop('openaccess', axis=1) # убираем что-то, что непонятно откуда взялось и мешаеет
            temp3 = temp3.rename_axis(None).transpose() # переворачиваем
            temp3.columns = temp3.columns.get_level_values(0) # лечим MultiIndex
            temp4 = begin.merge(temp3,how='outer')
            temp4.index=['{}'.format(listc[alist.index(i)])]
            aresc = pd.concat([aresc,temp4])
            #* цитирования *#

            if (blist != []):
                k = blist[alist.index(i)]
                sb = ScopusSearch(k, subscriber = False)
                df2 = pd.DataFrame(pd.DataFrame(sb.results))
                df2['coverDate'] = pd.DatetimeIndex(df2['coverDate']).year # меняем данные на формат .year

                # публикации #
                temp5 = pd.DataFrame(df2.value_counts(subset=['coverDate']), columns=['count']) # считаем публикации
                temp5 = temp5.rename_axis(None).transpose() # переворачиваем, потому что костыли надо вертеть
                temp5.columns = temp5.columns.get_level_values(0) # лечим то, что программа считает, что это мультииндекс
                temp6 = begin.merge(temp5,how='outer') #объединяем результаты
                temp6.index=['{}'.format(listc[blist.index(k)])] # подписываем конференцию
                bresp = pd.concat([bresp,temp6])
                #* публикации *#

                # цитирования #
                temp7 = df2.groupby('coverDate').sum('citedby_count').drop('openaccess', axis=1) # убираем что-то, что непонятно откуда взялось и мешаеет
                temp7 = temp7.rename_axis(None).transpose() # переворачиваем
                temp7.columns = temp7.columns.get_level_values(0) # лечим MultiIndex
                temp8 = begin.merge(temp7,how='outer')
                temp8.index=['{}'.format(listc[blist.index(k)])]
                bresc = pd.concat([bresc,temp8])
                    #* цитирования *#
    print('Publications (Italy):')
    display(aresp)
    print('Citations (Italy):')
    display(aresc)
    if (blist != []):
        print('Publications (World):')
        display(bresp)
        print('Citations (World):')
        display(bresc)
        
    display(downbut1, metbut)
        

def Download1(b):
    username = os.getlogin()
    
    if (blist != []):
        with pd.ExcelWriter(f'C:\\Users\\{username}\\Desktop\\output.xlsx') as writer:  
            aresp.to_excel(writer, sheet_name='Country (Pub)')
            aresc.to_excel(writer, sheet_name='Country (Cit)')
            bresp.to_excel(writer, sheet_name='World (Pub)')
            bresc.to_excel(writer, sheet_name='World (Cit)')
            
    else:
        with pd.ExcelWriter(f'C:\\Users\\{username}\\Desktop\\output.xlsx') as writer:  
            aresp.to_excel(writer, sheet_name='Country (Pub)')
            aresc.to_excel(writer, sheet_name='Country (Cit)')
        
    print('Downloaded to desktop!')
    
def metrics(b):
    TO_a = pd.DataFrame([aresp.sum(axis=1)], index=[f'Total output ({ac})']).transpose()
    TCS_a = pd.DataFrame([aresc.sum(axis=1)], index=[f'TCS ({ac})']).transpose()
    CPP_a = pd.DataFrame([TCS_a[f'TCS ({ac})']/TO_a[f'Total output ({ac})']], index=[f'CPP ({ac})']).transpose()
    A_res = pd.merge(pd.merge(TO_a,TCS_a,left_index=True, right_index=True),CPP_a,left_index=True, right_index=True)
    
    metres = A_res
    
    
    
    if (b != []):
        TO_b = pd.DataFrame([bresp.sum(axis=1)], index=['Total output (World)']).transpose() # Total output
        TCS_b = pd.DataFrame([bresc.sum(axis=1)], index=['TCS (World)']).transpose()
        CPP_b = pd.DataFrame([TCS_b['TCS (World)']/TO_b['Total output (World)']], index=['CPP (World)']).transpose()
        B_res = pd.merge(pd.merge(TO_b,TCS_b,left_index=True, right_index=True),CPP_b,left_index=True, right_index=True)
        
        MNCS_ci = aresc/aresp
        MNCS_ei = bresc/bresc
        MNCS_sum = (MNCS_ci/MNCS_ei).sum(axis=1)
        MNCS_list = MNCS_sum*0.1
        MNCS_a = pd.DataFrame([MNCS_list], index=[f'MNCS ({ac})']).transpose()

        A_res = pd.merge(A_res, MNCS_a, left_index=True, right_index=True)
        
        metres = pd.merge(A_res, B_res, left_index=True, right_index=True)
        metres.sort_values(f'MNCS ({ac})')
        
        
    
    display(metres)
    
    print('\033[1m' + 'Total output' + '\033[0m' + ' — total count of publications', 
          '\033[1m' + 'TCS' + '\033[0m' + ' — total count of citations',
          '\033[1m' + 'CPP' + '\033[0m' + ' — citation per paper',
          sep='\n')
    
    if (b != []):
        print('\033[1m' + 'MNCS' + '\033[0m' + ' — Mean normalized citation score\n\n',
              '\033[1m' + 'If MNCS is >1, the conference is a good choice to be noticed' + '\033[0m', sep='')
    
############################################# КНОПКИ

# кнопка для выбора файла
browsebut = Button(description="Browse file...", layout=Layout(width='50%', height='80px'))
gobut = Button(description="Let's search!", layout=browsebut.layout)
downbut1 = Button(description="Download as an Excel file", layout=browsebut.layout)
metbut = Button(description="Get metrics", layout=browsebut.layout)

# кнопка "я закончил"
ready = widgets.ToggleButton(description='OK', disabled=False, button_style='success', icon='check', layout=browsebut.layout)
ready2 = widgets.ToggleButton(description='OK', disabled=False, button_style='success', icon='check', layout=browsebut.layout)
ready3 = widgets.ToggleButton(description='OK', disabled=False, button_style='success', icon='check', layout=browsebut.layout)

years = [i for i in range (1999, 2022)]
pys = SelectionRangeSlider(options=years, description='Years', layout=browsebut.layout)

acs = Text(placeholder='If not needed, leave blank', description='Country:', layout=browsebut.layout)

############################################ КНОПКИ ЗАКОНЧИЛИСЬ

browsebut.on_click(select_files) # по нажатию — выбираем файл
    
display(browsebut, ready) # показываем две кнопки

ready.observe(on_click1, 'value')

gobut.on_click(ScSearch)

downbut1.on_click(Download1)

metbut.on_click(metrics)

Button(description='Browse file...', layout=Layout(height='80px', width='50%'), style=ButtonStyle())

ToggleButton(value=False, button_style='success', description='OK', icon='check', layout=Layout(height='80px',…